### Generator

In [ ]:
from src.rag.components.generator import LLamaCppGeneratorComponent

In [ ]:
prompt = "you are a helpful mortgage advisor"

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [ ]:
llama_cpp_generator._ping_api()

### Reading the question and Paragraphs

In [ ]:
from pathlib import Path

In [ ]:
current_working_directory = Path.cwd()

In [ ]:
data_directory = current_working_directory.joinpath("datasets/halifax_intermediaries/")
data_directory.exists()

In [ ]:
embedding_model_name = "intfloat/multilingual-e5-large"

In [ ]:
path  = data_directory.joinpath(f"question_and_paragraph_{embedding_model_name.split('/')[-1]}.csv")

In [ ]:
path

In [ ]:
import pandas as pd

In [ ]:
question_paragraph = pd.read_csv(path, index_col=[0, 1])

In [ ]:
question_paragraph

In [ ]:
question_paragraph.to_clipboard()

In [ ]:
from tqdm import tqdm

generated_answers = {}
for question in tqdm(question_paragraph.index.levels[0], desc="Generating answers"):
    paragraphs = question_paragraph.loc[question].content.tolist()
    template_input = {
        "documents": paragraphs,
        "question": question
    }
    response = llama_cpp_generator.run(template_values=template_input)
    generated_answers[question] = response

In [ ]:
len(generated_answers)

In [ ]:
import json

In [ ]:
loaded_answers = {}
for question in generated_answers.keys():
    answer = json.loads(generated_answers[question])
    loaded_answers[question] = answer

In [ ]:
generate_answers_df = pd.DataFrame.from_dict(loaded_answers, orient='index')

In [ ]:
generate_answers_df

In [ ]:
loaded_answers.get("Can you let me know which order i will need to do a product transfer and a further advance for existing client ?")

In [ ]:
generate_answers_df.text = generate_answers_df.text.str.replace("\n", "").str.strip()

In [ ]:
failed_question_df = generate_answers_df[generate_answers_df.is_correct == False]

In [ ]:
failed_question_df

### Tommorow we need to investigate the failed question and try different appraoches:

- Question rephrasing
- Drop nan in question
- Revisit the question and correct the questions.

In [ ]:
### Multiple Query Retrieval

In [ ]:

failed_question_df.to_csv(current_working_directory.joinpath(f"failed_questions_{embedding_model_name.split('/')[-1]}.csv"))

In [ ]:
rephraser_prompt = """ 
"{{question}}"
Given the above question, rephrase and expand it to help you do better answering. 
 Generates at least 3 different questions that are in the original question 
 Maintain all information in the original question.
 But fix any typos and replace abbreviation by full text
 The goal of those questions is to retrieve better chunks using embedding search"""

Tomorrow try RAG function approach.